# <img style="float: left; padding-right: 0.7cm; width: 3.0cm" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/179px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"> IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2022-1**<br>
**Profesora:** Paula Aguirre <br>
**Integrantes:** Rodolfo Cruz, Nicolas San Martin, Benjamin Santander, Daniela Victorero <br>

----

 <h1><center>Proyecto: Análisis sobre la insuficiencia del desarrollo educacional con relación al rendimiento académico y otras variables.</center></h1>

## Contexto y motivación:

Deseamos saber si existe una relación entre la situación socioeconómica del individuo
y su capital cognitivo, para tomar medidas que permitan mejorar y equiparar el desarrollo
cognitivo del individuo, así como su acceso considerando el contexto de reformas
educacionales y pandemia. En primer lugar, se define capital cognitivo como “la suma de los saberes adquiridos a
consecuencia del capital cultural de origen más las oportunidades educacionales formales”
(Rosas, 2013, pág. 55).
En segundo lugar, según Pierre Bourdieu, se define capital cultural como “al conjunto
de bienes culturales disponibles, transmitidos de generación en generación, los que explican
las diferencias de los resultados escolares y en la ubicación relativa en la pirámide social.”
(Rosas, 2013, pág. 15). Por lo tanto, creemos que sería interesante compararar el rendimiento escolar de los estudiantes en funcion de ciertos indicadores socioeconomicos tales como:

- Indicador de calidad educacional.
- Indicados de nivel socioeconómico.




## Preguntas objetivo: 

Las preguntas anteriores eran:
1. ¿Es la educación una inversión al largo plazo?
2. ¿Existe una relación entre el nivel socioeconómico del estudiante y capital cognitivo?
3. ¿Influye la ubicación en el rendimiento académico?
4. ¿Cuáles son las regiones con mayor rendimiento académico?
5. ¿Es acaso la inversión al estudiante un factor que determine el éxito académico?

A lo largo del proyecto, se ha decidido enfocar la pregunta a que si existe una relacion entre el nivel socioeconomico del estudiante y su capital congnitivo, debido a la cantidad de informacion presente respecto al tema y tambien considerando las conclusiones obtenidas de la tarea 3 en cuando a la correlacion existente entre variables para un mejor desempeño.


## Datos: 
describa los datos utilizados y su origen, y documente los procedimientos de recolección, preparación y transformación de datos.

## Análisis exploratorio de datos: 
describa el proceso de exploración de sus datos, incluyendo visualizaciones, análisis gráficos, análisis estadísticos, etc. Explique los resultados y conclusiones preliminares obtenidas, y cómo estos influyen o motivan el método de modelamiento elegido. Justifique todas las decisiones adoptadas.



## Análisis de datos: 
describa y documente los métodos computacionales y estadísticos aplicados al modelamiento de sus datos. Justifique las elecciones y decisiones adoptadas (ej: selección de variables y parámetros, configuración de algoritmos, normalizaciones, validaciones, etc.), y evalúe sus resultados en base a métricas apropiadas al modelo elaborado (Ej. error cuadrático, intervalo de confiabilidad, precisión, recall, F1, matriz de confusión, etc.)

## Resumen de los resultados: 
explique narrativa y visualmente sus resultados, cómo estos responden a la pregunta original, y las posibles limitaciones o sesgos de sus datos o análisis.

----

## Desarrollo

Como grupo pensamos que podria ser util enfocar el estudio del rendimiento estudiantil para la RM, asi luego extrapolar las mismas ideas a un panorama general de todas las regiones en Chile. Por lo tanto, comenzaremos analizando la distribucion de los establecimientos educacionales en santiago de acuerdo a indicadores socioeconomicos/sociodemograficos.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, Normalizer
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
import matplotlib.pyplot as plt
import matplotlib as mlp
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsRegressor
import geopandas as gpd
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from os import path
import warnings
try:
    import zipfile36
except:
    print('Instalar zipfile36: pip install zipfile36')
warnings.filterwarnings('ignore')

In [ ]:
data = gpd.read_file("C2017-2012_RM_pob_socioec.gpkg")
data

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
data.plot(ax = ax)
ax.set_title("Distribucion de población en Santiago según CENSO 2012-2018")

In [ ]:
geometry = data["geometry"]
data.drop("geometry", axis = 1, inplace=True)

In [ ]:
scaler = StandardScaler()
data_sc = scaler.fit_transform(data)
data_sc

In [ ]:
inertia=[]
kmax=12
for i in range(1,kmax):

    kmeans = KMeans(n_clusters=i,n_init=10)
    kncl= kmeans.fit(data_sc)

    print(i,kncl.inertia_)
    inertia.append(kncl.inertia_)

In [ ]:
f, ax = plt.subplots(1,1, figsize=(20,10))

ax.plot(range(1,kmax),inertia,marker='o',ls='--')
ax.set_xlabel('N Clusters')
ax.set_ylabel('Inercia de clusters')
ax.set_title('K-means - Método del codo')

for i in range(1,len(inertia)):
    delta=(-inertia[i-1]+inertia[i])/inertia[i-1]*100
    ax.text(i+1.1,inertia[i],'%.2f'%delta+'%',color='b')
ax.set_xticks(range(1,kmax))

plt.show()

In [ ]:
k=5

# Inicializamos la instancia  KMeans 
kmeans = KMeans(n_clusters=k,random_state=0)


# Correr el algoritmo sobre el conjunto de variable normalizadas
kcls = kmeans.fit(data_sc)

In [ ]:
data['kcls_labels'] = kcls.labels_
centers = kcls.cluster_centers_
centers_i = scaler.inverse_transform(centers)
centers_i

In [ ]:
data["geometry"] = geometry

In [ ]:
data.plot(figsize=(20,15),column = "kcls_labels", categorical = True, legend = True)

In [ ]:
values = data.groupby(by = "kcls_labels").mean().sort_values(by="NSE")
data.groupby(by = "kcls_labels").mean().sort_values(by="NSE")

In [ ]:
colegios = pd.read_csv("Directorio_Oficial_EE_2021.csv", sep=";")
colegios.info()

In [ ]:
colegios[colegios["NOM_COM_RBD"] == "PUENTE ALTO"]

In [ ]:
colegios["LONGITUD"] = colegios["LONGITUD"].replace(" ", np.nan)
colegios["LATITUD"] = colegios["LATITUD"].replace(" ", np.nan)
colegios.dropna()
colegios["LONGITUD"] = colegios["LONGITUD"].str.replace(",",".").astype(np.float64)
colegios["LATITUD"] = colegios["LATITUD"].str.replace(",",".").astype(np.float64)
colegios = colegios[colegios["LONGITUD"] <= -50]

In [ ]:
localizaciones = gpd.GeoDataFrame(colegios, geometry= gpd.points_from_xy(colegios["LONGITUD"], colegios["LATITUD"]), crs = 4326)

In [ ]:
codigos = [13101, 13102, 13103, 13104, 13105, 13106, 13107, 13108, 13109, 13110, 13111, 13112, 13113, 13114, 13115, 13116, 13117, 13118, 13119, 13120, 13121, 13122, 13123, 13124, 13125, 13126, 13127, 13128, 13129, 13130, 13131, 13132, 13201, 13202, 13401, ]
localizaciones = localizaciones[localizaciones["COD_COM_RBD"].isin(codigos) == True]

In [ ]:
data = data.to_crs(4326)

In [ ]:
palette = {'GRATUITO': 'violet', '$1.000 A $10.000': 'orange', '$10.001 A $25.000' : 'yellow', '$25.001 A $50.000' : 'black', '$50.001 A $100.000' : 'red', 'MAS DE $100.000' : 'pink', "SIN INFORMACION":"purple"}
cmap = mlp.colors.ListedColormap([palette[b] for b in localizaciones['PAGO_MENSUAL'].unique()])

fig , ax= plt.subplots(2, 1, figsize=(40, 30))
plot_mapa = data.plot(ax = ax[0],column = "kcls_labels", categorical = True, legend = True)
localizaciones.plot(ax = plot_mapa, markersize=6, color = "black")
ax1 = data.plot(ax = ax[1], color = "gray")
localizaciones.plot(ax = ax1, markersize=6, cmap=cmap ,categorical = True, column="PAGO_MENSUAL", legend=True)

Con el grafico anterior podemos observar la distribucion de los establecimientos educacionales en relacion a los indicadores socioeconomicos / sociodemograficos. De esto es interesante descubrir ciertas relaciones de cantidad establecimientos v/s percepcion de calidad de vida. Aprovechando las columnas del dataset tambien podemos agregar el valor de pago mensual de cada estableciento. Tambien encontramos una relacion entre la red vial del Gran Santiago y el acceso a los establecimientos, donde alejados de las grandes carreteras, encontramos mayor cantidad de colegios gratuito a excepcion de la comuna de Providencia/Las Condes/Lo Barnechea/ Vitacura.

In [ ]:
data.groupby("kcls_labels").mean()

Ahora investigaremos el rendimiento de los estudiantes de cada comuna de santiago en funcion de la jornada escolar que cumplen, es decir, si van por la manana (1), por la tarde (2) o jornada completa (3).

In [ ]:
# si queremos el archivo csv cambiamos esta variable
crear_archivo = False

In [ ]:
df_2 = pd.read_csv('promedios.zip',compression='zip', sep=';')
dic = {}
for i in df_2.columns:
    dic[i] = i.upper()
df_2.rename(dic, axis=1, inplace=True)
df_2[f'PROM_GRAL'] = df_2[f'PROM_GRAL'].replace(',', '.', regex=True)
df_2[f'PROM_GRAL'] = df_2[f'PROM_GRAL'].astype(np.float64)
df_2 = df_2[df_2[f'PROM_GRAL'] != 0]
df_2 = df_2[['MRUN', 'PROM_GRAL','ASISTENCIA', 'SIT_FIN_R','COD_COM_RBD', 'COD_JOR', 'NOM_COM_RBD']]
df_2['COD_JOR'] = df_2['COD_JOR'].astype(np.int64)
df_2 = df_2[(df_2['COD_JOR'] == 1) | (df_2['COD_JOR'] == 2) | (df_2['COD_JOR'] == 3)]
comunas = df_2.copy()
if crear_archivo:
    df_2.to_csv(f'jornadas20', index=False)



In [ ]:
df_2 = pd.read_csv('promedios.zip',compression='zip', sep=';')
dic = {}
for i in df_2.columns:
    dic[i] = i.upper()
df_2.rename(dic, axis=1, inplace=True)
df_2[f'PROM_GRAL'] = df_2[f'PROM_GRAL'].replace(',', '.', regex=True)
df_2[f'PROM_GRAL'] = df_2[f'PROM_GRAL'].astype(np.float64)
df_2 = df_2[df_2[f'PROM_GRAL'] != 0]
df_2 = df_2[['MRUN', 'PROM_GRAL','ASISTENCIA', 'SIT_FIN_R','COD_REG_RBD', 'COD_JOR', 'NOM_REG_RBD_A']]
df_2['COD_JOR'] = df_2['COD_JOR'].astype(np.int64)
df_2 = df_2[(df_2['COD_JOR'] == 1) | (df_2['COD_JOR'] == 2) | (df_2['COD_JOR'] == 3)]
regiones = df_2.copy()
if crear_archivo:
    df_2.to_csv(f'jornadas_region20', index=False)


In [ ]:
codigos = [13101, 13102, 13103, 13104, 13105, 13106, 13107, 13108, 13109, 13110, 13111, 13112, 13113, 13114, 13115, 13116, 13117, 13118, 13119, 13120, 13121, 13122, 13123, 13124, 13125, 13126, 13127, 13128, 13129, 13130, 13131, 13132, 13201, 13202, 13401]

In [ ]:
fig, ax = plt.subplots(20, 3, figsize=(20, 100))
ax = ax.flatten()
largo = len(codigos)
axis = 0
df_plot = comunas
for k in range(largo):
    plot = df_plot[(df_plot['COD_COM_RBD'] == codigos[k])] # seleccionamos la comuna
    for j in range(1,4):
        try:
            plot_final = plot[(df_plot['COD_JOR'] == j)] # elegimos la jornada
            plot_final = plot_final.groupby('PROM_GRAL').count()
            axes = sns.scatterplot(data=plot_final, x='PROM_GRAL', y='COD_JOR',ax=ax[axis], legend=True)
            axes.set_title(f"{plot['NOM_COM_RBD'].iloc[0]} jornada: {j}", fontsize=25)
            if j == 1:
                axes.set_title(f"{plot['NOM_COM_RBD'].iloc[0]} jornada: Mañana", fontsize=11)
            elif j == 2:
                axes.set_title(f"{plot['NOM_COM_RBD'].iloc[0]} jornada: Tarde", fontsize=11)
            else:
                axes.set_title(f"{plot['NOM_COM_RBD'].iloc[0]} jornada: Mañana/Tarde", fontsize=11)

            axis += 1
        except:
            pass
plt.show()

In [ ]:
fig, ax = plt.subplots(16, 3, figsize=(20, 100))
ax = ax.flatten()
largo = len(codigos)
axis = 0
df_plot = regiones
for k in range(1,17):
    plot = df_plot[(df_plot['COD_REG_RBD'] == k)] # seleccionamos la comuna
    for j in range(1,4):
        try:
            plot_final = plot[(df_plot['COD_JOR'] == j)] # elegimos la jornada
            plot_final = plot_final.groupby('PROM_GRAL').count()
            axes = sns.scatterplot(data=plot_final, x='PROM_GRAL', y='COD_JOR',ax=ax[axis], legend=True)
            if j == 1:
                axes.set_title(f"{plot['NOM_REG_RBD_A'].iloc[0]} jornada: Mañana", fontsize=15)
            elif j == 2:
                axes.set_title(f"{plot['NOM_REG_RBD_A'].iloc[0]} jornada: Tarde", fontsize=15)
            else:
                axes.set_title(f"{plot['NOM_REG_RBD_A'].iloc[0]} jornada: Mañana/Tarde", fontsize=15)
            axis += 1
        except:
            pass

plt.show()

Cluster

In [ ]:
datos = pd.read_csv("20220617_NEM_PERCENTILES_JOVENES_2021_20220530_PUBL.csv", sep=";")

In [ ]:
datos["RBD"] = datos["RBD"].astype("category")
datos["NEM"] = datos["NEM"].str.replace(",",".")
datos["NEM"] = datos["NEM"].astype(np.float64)


In [ ]:
agrupacion_promedios = datos.groupby(by = "RBD").mean().reset_index()

In [ ]:
directorio = pd.read_csv("Directorio_Oficial_EE_2021.csv", sep=";")

In [ ]:
directorio = directorio[directorio["NOM_RBD"].str.contains("JARDIN+|SALA CUNA+", regex=True) == False]

In [ ]:
directorio["LONGITUD"] = directorio["LONGITUD"].replace(" ", np.nan)
directorio["LATITUD"] = directorio["LATITUD"].replace(" ", np.nan)
directorio.dropna()
directorio["LONGITUD"] = directorio["LONGITUD"].str.replace(",",".").astype(np.float64)
directorio["LATITUD"] = directorio["LATITUD"].str.replace(",",".").astype(np.float64)


In [ ]:
localizaciones = gpd.GeoDataFrame(directorio, geometry= gpd.points_from_xy(directorio["LONGITUD"], directorio["LATITUD"]), crs=4326)

In [ ]:
conjunto = localizaciones.merge(agrupacion_promedios, how = "left", on = "RBD")

In [ ]:
datos_compacto_2021 = conjunto[['AGNO', 'RBD', 'NOM_RBD', 'COD_REG_RBD', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', "COD_DEPE2", 'NEM', 'geometry']]

In [ ]:
datos_compacto_2021 = datos_compacto_2021[datos_compacto_2021["geometry"] != None]


In [ ]:
ruta_cartografia = path.join("DivisionPoliticoAdministrativa2020", "COMUNA", "COMUNAS_2020.shp")
comunas = gpd.read_file(ruta_cartografia).to_crs(4326)
# y ploteamos los datos del geodataframe
plot_mapa = comunas.plot(color = "orange", figsize=(20, 50))
datos_compacto_2021.plot(ax = plot_mapa, markersize=3)
plot_mapa.set_title("Grafica de distribución de escuelas en Chile")

In [ ]:
geometry = datos_compacto_2021[["RBD", "NOM_RBD", "geometry"]]

In [ ]:
datos_compacto_2021.drop("geometry", axis = 1, inplace=True)

In [ ]:
subencion = pd.read_csv("20220509_Detalle Subvenciones 2021_20220413.csv", sep=";", encoding="ISO-8859-1")

In [ ]:
subencion = subencion.groupby("NOM_RBD").mean().reset_index()
subencion.drop(["AGNO", "COD_DEPE2", "COD_REG_RBD", "COD_PRO_RBD", "COD_COM_RBD"], axis = 1, inplace = True)

In [ ]:
subencion["RBD"] = subencion["RBD"].astype(np.int64)

In [ ]:
ingreso = pd.read_csv("Ingresos_medios_2010-2021.csv", sep=",")

In [ ]:
ingreso = ingreso[ingreso["agno"] == 2021]

In [ ]:
ingreso = ingreso[ingreso["agno"] == 2021]

In [ ]:
ingreso.rename(columns={"region":"COD_REG_RBD"}, inplace = True)

In [ ]:
ingreso = ingreso[["COD_REG_RBD", "ingre_medio"]]

In [ ]:
resumen_docente = pd.read_csv("Dotacion_docente_2021.csv", sep=";")[["RBD", "NOM_RBD", "DC_TOT"]]

In [ ]:
evaluacion = pd.read_csv("20201104_SNED_2020_2021.csv", sep=";")[["RBD", "NOM_RBD", "EFECTIVR", "MEJORAR", "IGUALDR"]]

In [ ]:
evaluacion.replace(" ", 0, inplace=True)

In [ ]:
evaluacion["EFECTIVR"]= evaluacion["EFECTIVR"].str.replace(",",".").astype(np.float64)
evaluacion["MEJORAR"] = evaluacion["MEJORAR"].str.replace(",",".").astype(np.float64)
evaluacion["IGUALDR"]= evaluacion["IGUALDR"].str.replace(",",".").astype(np.float64)

##### Agregamos los datos ajustados

In [ ]:
datos_compacto_2021 = datos_compacto_2021.merge(subencion, how="left", on = ["RBD", "NOM_RBD"])

In [ ]:
datos_compacto_2021 = datos_compacto_2021.merge(ingreso, how="left", on = "COD_REG_RBD")

In [ ]:
datos_compacto_2021 = datos_compacto_2021.merge(resumen_docente, how="left", on = ["RBD", "NOM_RBD"])

In [ ]:
datos_compacto_2021 = datos_compacto_2021.merge(evaluacion, how="left", on = ["RBD", "NOM_RBD"])

###### Limpieza de datos para el cluster kmeans

In [ ]:
datos_compacto_2021 = datos_compacto_2021.dropna()

In [ ]:
datos_compacto_2021 = datos_compacto_2021[['AGNO', 'RBD', 'NOM_RBD', 'COD_REG_RBD', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPE2', 'NEM', "subv_normal", "ingre_medio", "DC_TOT", "EFECTIVR", "MEJORAR", "IGUALDR"]]

In [ ]:
scaler = StandardScaler()
data_sc = scaler.fit_transform(datos_compacto_2021[["AGNO", "COD_DEPE2", "NEM", "subv_normal", "ingre_medio", "DC_TOT", "EFECTIVR", "MEJORAR", "IGUALDR"]])
data_sc

In [ ]:
inertia=[]
kmax=20
for i in range(1,kmax):

    kmeans = KMeans(n_clusters=i,n_init=10)
    kncl= kmeans.fit(data_sc)

    print(i,kncl.inertia_)
    inertia.append(kncl.inertia_)

In [ ]:
f, ax = plt.subplots(1,1, figsize=(20,10))

ax.plot(range(1,kmax),inertia,marker='o',ls='--')
ax.set_xlabel('N Clusters')
ax.set_ylabel('Inercia de clusters')
ax.set_title('K-means - Método del codo')

for i in range(1,len(inertia)):
    delta=(-inertia[i-1]+inertia[i])/inertia[i-1]*100
    ax.text(i+1.1,inertia[i],'%.2f'%delta+'%',color='b')
ax.set_xticks(range(1,kmax))

plt.show()

In [ ]:
k=7

# Inicializamos la instancia  KMeans 
kmeans = KMeans(n_clusters=k,random_state=0)


# Correr el algoritmo sobre el conjunto de variable normalizadas
kcls = kmeans.fit(data_sc)


In [ ]:
datos_compacto_2021['kcls_labels'] = kcls.labels_

In [ ]:
centers = kcls.cluster_centers_
centers_i = scaler.inverse_transform(centers)

centers_i

In [ ]:
datos_compacto_2021 = datos_compacto_2021.merge(geometry, on = ["RBD", "NOM_RBD"])


In [ ]:
datos_compacto_2021 = gpd.GeoDataFrame(datos_compacto_2021)

In [ ]:
ruta_cartografia = path.join("DivisionPoliticoAdministrativa2020", "COMUNA", "COMUNAS_2020.shp")
comunas = gpd.read_file(ruta_cartografia).to_crs(4326)
# y ploteamos los datos del geodataframe
plot_mapa = comunas.plot(color = "orange", figsize=(20, 50))
datos_compacto_2021.plot(ax = plot_mapa, column = "kcls_labels", categorical = True, legend = True)
plot_mapa.set_title("Grafica de distribución de escuelas en Chile")



##### Obtenemos los promedios de los cluster

In [ ]:
values = datos_compacto_2021.groupby(by = "kcls_labels").mean().sort_values(by="NEM")[["AGNO", "COD_DEPE2", "NEM", "subv_normal", "ingre_medio", "DC_TOT", "EFECTIVR", "MEJORAR", "IGUALDR"]]
values

Análisis de ingreso a universidades con 7 años de acreditación en las comunas de Santiago.

In [ ]:
ruta_comunas = path.join("DivisionPoliticoAdministrativa2020", "COMUNA", "COMUNAS_2020.shp")
comunas = gpd.read_file(ruta_comunas)
comunas['COMUNA'] = comunas['COMUNA'].astype('object')
#https://stackoverflow.com/questions/39512002/convert-whole-dataframe-from-lower-case-to-upper-case-with-pandas
comunas['COMUNA'] = comunas['COMUNA'].str.upper()
comunas = comunas.set_index('COMUNA')
comunas

In [ ]:
rendimiento_2021 = pd.read_csv('promedios.zip',compression='zip', sep=';')
superior_2022 = pd.read_csv('superior_2022.csv', delimiter= ';')

In [ ]:
rendimiento_2021 = rendimiento_2021.dropna(subset = ['MRUN'])
rendimiento_2021['EDAD_ALU'] = rendimiento_2021['EDAD_ALU'].replace(' ', np.nan)
rendimiento_2021 = rendimiento_2021.dropna(subset = ['EDAD_ALU'])
rendimiento_2021['EDAD_ALU'] = rendimiento_2021['EDAD_ALU'].astype(np.int64)
rendimiento_2021['EDAD_ALU'] = rendimiento_2021['EDAD_ALU'].astype(np.int64)
rendimiento_2021 = rendimiento_2021[rendimiento_2021['EDAD_ALU'] <= 17]
rendimiento_2021 = rendimiento_2021[['MRUN', 'AGNO', 'COD_REG_RBD', 'COD_PRO_RBD', 'NOM_COM_RBD']]
rendimiento_2021

In [ ]:
superior_2022 = superior_2022.rename(columns={'mrun' : 'MRUN'})
superior_2022 = superior_2022.dropna(subset = ['MRUN'])
superior_2022 = superior_2022[['MRUN', 'codigo_unico',  'cod_inst', 'nomb_inst', 'cod_carrera', 'nomb_carrera', 'region_sede', 'provincia_sede', 'comuna_sede', 'valor_matricula', 'valor_arancel', 'codigo_demre', 'acre_inst_anio']]
superior_2022


In [ ]:
ingreso = rendimiento_2021.merge(superior_2022, on = 'MRUN')
ingreso = ingreso.rename(columns = {'NOM_COM_RBD' : 'COMUNA'})
ingreso

In [ ]:
lista_universidades_7 = ['PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE', 'UNIVERSIDAD DE CHILE', 'UNIVERSIDAD DE SANTIAGO DE CHILE']
for universidad in lista_universidades_7:
    ub_colegios_universidad = ingreso[ingreso['nomb_inst'] == universidad]
    alu_por_comuna = ub_colegios_universidad.groupby('COMUNA').count()
    comunas[universidad] = alu_por_comuna['MRUN']
comunas = comunas.replace(np.nan, float(0))
comunas

In [ ]:
for universidad in lista_universidades_7:
    if universidad != 'UNIVERSIDAD DE CONCEPCION':
        fig, ax = plt.subplots()
        prov_stgo = comunas[comunas['PROVINCIA'] == 'Santiago']
        prov_stgo.plot(column = universidad, legend = True, ax = ax)
        bounds = prov_stgo.geometry.total_bounds
        xlim = ([bounds[0],  bounds[2]])
        ylim = ([bounds[1],  bounds[3]])
        plt.title(universidad)

In [ ]:
ub_colegios = directorio

ub_colegios = ub_colegios[ub_colegios['LONGITUD'] != ' ']
ub_colegios = ub_colegios[ub_colegios['LATITUD'] != ' ']
ub_colegios['LONGITUD'] = ub_colegios['LONGITUD'].str.replace(',', '.')
ub_colegios['LATITUD'] = ub_colegios['LATITUD'].str.replace(',', '.')
ub_colegios = ub_colegios[ub_colegios['PAGO_MENSUAL'] != 'SIN INFORMACION']
ub_colegios['LONGITUD'] = ub_colegios['LONGITUD'].astype(np.float64)
ub_colegios['LATITUD'] = ub_colegios['LATITUD'].astype(np.float64)
ub_colegios['MAT_TOTAL'] = ub_colegios['MAT_TOTAL'].astype(np.float64)
ub_colegios['MATRICULA'] = ub_colegios['MATRICULA'].astype(np.float64)
ub_colegios = ub_colegios.dropna()
ub_colegios = ub_colegios.rename(columns = {'NOM_COM_RBD':'COMUNA'})
ub_colegios = gpd.GeoDataFrame(ub_colegios, geometry=gpd.points_from_xy(ub_colegios['LONGITUD'], ub_colegios['LATITUD']))
ub_colegios

In [ ]:
for universidad in lista_universidades_7:
    if universidad != 'UNIVERSIDAD DE CONCEPCION':
        fig, ax = plt.subplots(figsize = (30, 20))
        prov_stgo = comunas[comunas['PROVINCIA'] == 'Santiago']
        prov_stgo.plot(column = universidad, legend = True, ax = ax)
        ub_colegios_stgo = ub_colegios[ub_colegios['COD_PRO_RBD'] == 131]
        ub_colegios_stgo.plot(column = 'PAGO_MENSUAL', categorical = True, ax = ax, legend = True, s = 40)
        bounds = prov_stgo.geometry.total_bounds
        xlim = ([bounds[0],  bounds[2]])
        ylim = ([bounds[1],  bounds[3]])
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        plt.title(universidad)